## How to generate text: using different decoding methods for language generation with Transformers

In [ ]:
%pip install -q transformers

In [5]:
import math
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch


In [9]:
print(torch.device("cuda"))

cuda


In [ ]:


model_name_or_path = "../models/Wizard-Vicuna-7B-Uncensored-GPTQ"

torch_device = "cuda" if torch.cuda.is_available() else "cpu"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main").to(torch_device)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:

'''
model_inputs = tokenizer(prompt_template, return_tensors='pt').to(torch_device)

### Greedy Search
![greedy_search](../images/greedy_search.png)

In [4]:
# encode context the generation is conditioned on


# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=95)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=False))

Output:
----------------------------------------------------------------------------------------------------
<s> ### instruction : what is the outcome of pitcher 663432 pitching to batter 596115 ### input : Top of the 1 inning with 2 outs ### output : 663432 throws a 87 miles per hour Slider for a strike, 663432 throws a 86 miles per hour Slider for a strike, 663432 throws a 86 miles per hour Slider for a ball, 663432 throws a 86 miles per hour Slider for a strike and 596115 strikeout, event : strikeout, strikeout, event : strikeout, event : strikeout, strikeout, event : strikeout, strikeout, event : strikeout, play, event : strikeout double play, event : strikeout double play, event : strikeout double play, event


### Beam search
![beam_search](../images/beam_search.png)


In [5]:
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=65,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
### instruction : what is the outcome of pitcher 663432 pitching to batter 596115 ### input : Top of the 1 inning with 2 outs ### output : 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for a ball, 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider


In [63]:
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=65,
    num_beams=5,
    no_repeat_ngram_size=10,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
### instruction : what is the outcome of pitcher 663432 pitching to batter 596115 ### input : Top of the 8 inning with 2 outs ### output : 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for for strike, 663432 throws a 88 miles per hour Cutter for a ball, 663432 throws a 88 miles per hour Slider and 596115 hits into a field out, event : field out, event : field out, out, event : field out


In [69]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=65,
    num_beams=5,
    no_repeat_ngram_size=10,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: ### instruction : what is the outcome of pitcher 663432 pitching to batter 596115 ### input : Top of the 8 inning with 2 outs ### output : 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for for strike, 663432 throws a 88 miles per hour Cutter for a ball, 663432 throws a 88 miles per hour Slider and 596115 hits into a field out, event : field out, event : field out, out, event : field out
1: ### instruction : what is the outcome of pitcher 663432 pitching to batter 596115 ### input : Top of the 8 inning with 2 outs ### output : 663432 throws a 88 miles per hour Slider for a strike, 663432 throws a 88 miles per hour Slider for for strike, 663432 throws a 88 miles per hour Changeup for a ball, 663432 throws a 88 miles per hour Slider and 596115 hits into a field out, event : field out, event : field out, out, event : field out
2: 

### Sampling
![Sampling](../images/sampling_search.png)

In [ ]:
from transformers import set_seed
set_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

### Sampling w/Temperature
![Sampling](../images/sampling_search_with_temp.png)

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

### Top-K Sampling
![Top-K](../images/top_k_sampling.png)

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

### Top-p (nucleus) sampling
![Top-K](../images/top_p_sampling.png)

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))